In [28]:
import pandas as pd
import sqlite3
from sqlite3 import Error
 
def create_connection(file):
    conn = None
    try:
        conn = sqlite3.connect(file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn, string):
    cur = conn.cursor()  
    query = string
    rs = cur.execute(query)
    return rs.fetchall()

In [29]:
s1 = '''WITH y AS (SELECT DISTINCT x.facid,x.name, SUM(x.guest) OVER(PARTITION BY x.facid) AS gsum,
SUM(x.mem) OVER(PARTITION BY x.facid) AS msum FROM  (SELECT f.facid, f.name, 
CASE WHEN b.memid = 0 AND f.facid=b.facid
THEN b.slots*f.guestcost END AS guest,
CASE WHEN b.memid<> 0 AND f.facid = b.facid
THEN b.slots* f.membercost END as mem
FROM Facilities As f
INNER JOIN Bookings AS b ON f.facid=b.facid) AS x)

SELECT y.name, (y.gsum+y.msum) AS gross_rev FROM y
WHERE (y.gsum+y.msum) < 1000
ORDER BY gross_rev'''

s2 = '''SELECT a.surname||' '||a.firstname AS mem_name, b.surname||' '||b.firstname AS rec_name
FROM Members a, Members b
WHERE a.recommendedby = b.memid
ORDER BY mem_name'''

s3 = '''WITH x AS(SELECT facid,  sum(slots) AS tot_slots
FROM Bookings
WHERE memid <> 0
GROUP BY facid
ORDER BY sum(slots) DESC)

SELECT f.name, x.tot_slots
FROM Facilities AS f
INNER JOIN x ON f.facid=x.facid'''

s4 = '''
WITH x AS(SELECT facid, strftime(' %m ', date(starttime)) AS mon, sum(slots) as tot
FROM Bookings
WHERE memid <> 0
GROUP BY facid,mon)

SELECT f.name, x.mon AS 'month', x.tot AS tot_usage
FROM Facilities AS f
INNER JOIN x ON f.facid=x.facid'''

s = [s1,s2,s3,s4]

In [30]:
q1 = '''Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there a different cost for guests and members!'''

q2 = '''Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order'''

q3 = '''Q12: Find the facilities with their usage by member, but not guests'''

q4 = '''Find the facilities usage by month, but not guests'''

q = [q1,q2,q3,q4]

In [33]:
db = "sqlite_db_pythonsqlite.db"
conn = create_connection(db)
with conn: 
    for i,j in enumerate(s):
        print("\n\n",q[i],"\n  Query TASK ")
        df = pd.DataFrame(select_all_tasks(conn,j))
        print(df.head())

2.6.0


 Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there a different cost for guests and members! 
  Query TASK 
               0    1
0   Table Tennis  180
1  Snooker Table  240
2     Pool Table  270


 Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 
  Query TASK 
                0                1
0  Bader Florence  Stibbons Ponder
1      Baker Anne  Stibbons Ponder
2   Baker Timothy   Farrell Jemima
3      Boothe Tim       Rownam Tim
4  Butters Gerald     Smith Darren


 Q12: Find the facilities with their usage by member, but not guests 
  Query TASK 
                 0     1
0  Badminton Court  1086
1   Tennis Court 1   957
2   Massage Room 1   884
3   Tennis Court 2   882
4    Snooker Table   860


 Find the facilities usage by month, but not guests 
  Query TASK 
                0     1    2
0  Tennis Court 1   07   201
1  